### Module 25 - Required Activity 25.3 - Portfolio Project

##### DBN Submission (10/06/2025)

###### Description: This algorithm classifies satellite images from the EuroSAT database

###### EuroSAT Data Citation (Available at: https://github.com/phelber/eurosat [Accessed: 10/06/2025]):

    [1] Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
    
    @article{helber2019eurosat,
      title={Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification},
      author={Helber, Patrick and Bischke, Benjamin and Dengel, Andreas and Borth, Damian},
      journal={IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing},
      year={2019},
      publisher={IEEE}
    }
    [2] Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.
    
    @inproceedings{helber2018introducing,
      title={Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification},
      author={Helber, Patrick and Bischke, Benjamin and Dengel, Andreas and Borth, Damian},
      booktitle={IGARSS 2018-2018 IEEE International Geoscience and Remote Sensing Symposium},
      pages={204--207},
      year={2018},
      organization={IEEE}
    }

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

##### Step 1 - Load DataSet

In [4]:
# Load Data using Torch
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)])

trainset = torchvision.datasets.MNIST('data/eurosat', train = True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST('data/eurosat', train = False,
                                        download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)

100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:06<00:00, 1.42MB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 331kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:08<00:00, 191kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 8.73MB/s]


##### Step 2 - Define the CNN

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, name=None):
        super(Net, self).__init__()
        if name:
            self.name = name
        self.conv1 = nn.Conv2d(1, 10, (5,5))
        self.pool = nn.MaxPool2d((3,2), (2,2))
        self.conv2 = nn.Conv2d(10, 14, 5)
        self.fc1 = nn.Linear(14*6*1*2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 14*6*1*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net(name='LetNet5')
print(net)

LetNet5: total params: 35068
Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(3, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 14, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=168, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


##### Step 3 - Define Loss and Optimisation Functions

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

##### Step 4 - Train the Network

In [7]:
import time

start = time.time()

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

end = time.time()
print('training time ', end-start)

[1,  2000] loss: 1.513
[1,  4000] loss: 0.384
[1,  6000] loss: 0.298
[1,  8000] loss: 0.227
[1, 10000] loss: 0.188
[1, 12000] loss: 0.195
[1, 14000] loss: 0.172
[1, 16000] loss: 0.122
[1, 18000] loss: 0.136
[1, 20000] loss: 0.149
[1, 22000] loss: 0.113
[1, 24000] loss: 0.114
[1, 26000] loss: 0.122
[1, 28000] loss: 0.136
[1, 30000] loss: 0.142
[1, 32000] loss: 0.118
[1, 34000] loss: 0.105
[1, 36000] loss: 0.140
[1, 38000] loss: 0.117
[1, 40000] loss: 0.127
[1, 42000] loss: 0.118
[1, 44000] loss: 0.103
[1, 46000] loss: 0.130
[1, 48000] loss: 0.135
[1, 50000] loss: 0.100
[1, 52000] loss: 0.078
[1, 54000] loss: 0.138
[1, 56000] loss: 0.111
[1, 58000] loss: 0.093
[1, 60000] loss: 0.088
[2,  2000] loss: 0.081
[2,  4000] loss: 0.090
[2,  6000] loss: 0.083
[2,  8000] loss: 0.077
[2, 10000] loss: 0.075
[2, 12000] loss: 0.084
[2, 14000] loss: 0.077
[2, 16000] loss: 0.081
[2, 18000] loss: 0.088
[2, 20000] loss: 0.070
[2, 22000] loss: 0.077
[2, 24000] loss: 0.080
[2, 26000] loss: 0.067
[2, 28000] 

##### Step 5 - Test the Network on the Test Data

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %
